## OceanEddies GeoServer Example WFS Queries
Below are a few examples of how to use the WFS service of the OceanEddies Geoserver.

WFS is an open standard by the OGC. This notebook is meant as a quick intro rather than a full featured demo. Full tutorial for WFS and all other OGC webservices can be found at http://cite.opengeospatial.org/pub/cite/files/edu/index.html 

WFS was implemented in several versions. The GeoServer supports all major versions.


## Minimal demo
Below is an example of how to connect to the WFS and get data in the GeoJSON format. 

The example data is plottet on a leaflet map.

In [5]:
import json
import requests

from ipyleaflet import Map, basemaps, GeoJSON, LayersControl

In [9]:
m = Map(center=(17, -22), zoom=4, basemap=basemaps.Esri.NatGeoWorldMap)
m.add_control(LayersControl())
m

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{…

### web requests using `requests`

`requests` is a library for quering http/https service. We use it here to construct the query in a comprehensible manner.

In [16]:
wfs_base_url = "https://maps.geomar.de/geoserver/OceanEddies/wfs"
requested_layer = "OceanEddies:current_positions"
requests_parameters = {
                        'service': 'wfs',
                        'version': '1.0.0',
                        'request': 'getFeature',
                        'typename': requested_layer,
                        #The line below will filter the results on the server
                        #'CQL_FILTER': "type like 'WaveGlider'",  # OCG CQL filter
                        #'viewparams': "maxage:48",  # parameters for custom view on geoserver: set max age of position to 48h 
                        "outputformat": "application/json"
                      }
response = requests.get(wfs_base_url, params=requests_parameters)
print(f'use the following link for your query: {response.url}')

use the following link for your query: https://maps.geomar.de/geoserver/OceanEddies/wfs?service=wfs&version=1.0.0&request=getFeature&typename=OceanEddies%3Acurrent_positions&viewparams=maxage%3A1000&outputformat=application%2Fjson


In [17]:
all_positions = GeoJSON(data=response.json(), hover_style={'fillColor': 'red'}, name='Positions') 
m.add_layer(all_positions)

^^Scroll up to the the data on the map
### Server side filtering
WFS provides a couple of methods for filtering data on the server. The most simple one is `CQL_FILTER`. The basic 
principle is demonstrated below. In this case, we are filtering by the property `type`. `type` is the column that holds the device type (e.g. WaveGlider, OceanGlider, Drifter, ...) of a position. Other columns you
might be interested in filtering for could be `display_name` or `shortname`. Look into the `properties` of the returned geojson, each property was generated by a DB column you can filter by.

Scroll up again an uset the layer control widget after running the cell below.

In [18]:

requests_parameters = {
                        'service': 'wfs',
                        'version': '1.0.0',
                        'request': 'getFeature',
                        'typename': requested_layer,
                        #The line below will filter the results on the server
                        'CQL_FILTER': "type like 'WaveGlider'",  
                        "outputformat": "application/json"
                      }
response = requests.get(wfs_base_url, params=requests_parameters)
print(f'use the following link for your query: {response.url}')
wg_positions = GeoJSON(data=response.json(), hover_style={'fillColor': 'red'}, name='WaveGlider Positions') 
m.add_layer(wg_positions)
response.json()

use the following link for your query: https://maps.geomar.de/geoserver/OceanEddies/wfs?service=wfs&version=1.0.0&request=getFeature&typename=OceanEddies%3Acurrent_positions&CQL_FILTER=type+like+%27WaveGlider%27&outputformat=application%2Fjson


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'current_positions.fid--318c4a5d_16bb87a55bf_-7eb6',
   'geometry': {'type': 'Point', 'coordinates': [-20.91697, 21.16639]},
   'geometry_name': 'geom',
   'properties': {'shortname': 'WG_GMR4',
    'display_name': 'WG4',
    'name': 'Wave Glider GEOMAR4',
    'sensor_home': 'GEOMAR',
    'type': 'WaveGlider',
    'awi_urn': None,
    'surveydata_offering': None,
    'obs_timestamp': '2019-07-12T08:35:12Z',
    'obs_age': '-03:29:00',
    'heading': 218,
    'speed_over_ground': 0.202484,
    'platform_id': 2}}],
 'totalFeatures': 1,
 'numberMatched': 1,
 'numberReturned': 1,
 'timeStamp': '2019-07-12T12:04:29.801Z',
 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}}}

## Query using OWSLib
[OWSLib](https://geopython.github.io/OWSLib/) is a Python package for client programming with Open Geospatial Consortium (OGC) web service (hence OWS) interface standards, and their related content models.

If you are using python, OWSLib offers a convenient method to access data on the GeoServer.

In [19]:
from owslib.wfs import WebFeatureService
from owslib.fes import *
from owslib.etree import etree


In [20]:
wfs11 = WebFeatureService(url=wfs_base_url, version='1.1.0')
wfs11.identification.title

'GeoServer Web Feature Service'

In [21]:
list(wfs11.contents)

['OceanEddies:current_positions',
 'OceanEddies:eddy_positions',
 'OceanEddies:latest_positions']

In [22]:
filter = PropertyIsLike(propertyname='type', literal='WaveGlider', wildCard='*')
filterxml = etree.tostring(filter.toXML()).decode("utf-8")
response = wfs11.getfeature(typename='OceanEddies:latest_positions', filter=filterxml, outputFormat="application/json")
json_data = response.read()
json_data = json.loads(json_data)
json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'latest_positions.3',
   'geometry': {'type': 'Point', 'coordinates': [-15.36882, 27.99237]},
   'geometry_name': 'geom',
   'properties': {'shortname': 'WG_M',
    'display_name': 'WGM',
    'name': 'Wave Glider MARUM',
    'sensor_home': 'MARUM',
    'type': 'WaveGlider',
    'awi_urn': None,
    'surveydata_offering': None,
    'obs_timestamp': '2019-06-25T16:31:00Z',
    'obs_age': '-16 days -19:33:00',
    'heading': 24,
    'speed_over_ground': 1.781857}},
  {'type': 'Feature',
   'id': 'latest_positions.2',
   'geometry': {'type': 'Point', 'coordinates': [-20.91697, 21.16639]},
   'geometry_name': 'geom',
   'properties': {'shortname': 'WG_GMR4',
    'display_name': 'WG4',
    'name': 'Wave Glider GEOMAR4',
    'sensor_home': 'GEOMAR',
    'type': 'WaveGlider',
    'awi_urn': None,
    'surveydata_offering': None,
    'obs_timestamp': '2019-07-12T08:35:12Z',
    'obs_age': '-03:29:00',
    'heading': 218,
 

In [23]:
m = Map(center=(17, -22), zoom=4, basemap=basemaps.Esri.NatGeoWorldMap)
geo = GeoJSON(data=json_data, hover_style={'fillColor': 'red'}, name='Positions')
m.add_layer(geo)
m

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{…

## Server side storing of complex queries 
So far, we accessed the service using WFS version 1.0. Our Geoserver also supports newer WFS Versions.
WFS 2.0 introduced [stored queries](https://www.weichand.de/2012/04/22/wfs-2-0-stored-queries-beispiele/) which can be used to store complex queries on the server rather than having to build them on the client each time and then having to transmit them to the server. Might be very usefull to save bandwidth and reduce complexity of client code.